1. 문서 내용 읽기  
2. 문서 분할(쪼개기)  
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음.  
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐  
3. 임베딩 - 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달



In [ ]:
## 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain-community langchain-text-splitters langchain-chroma

In [ ]:
# %pip install pinecone langchain-pinecone

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 1. 문서 내용 읽고 분할 #############################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)
document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 #############################
   
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')


## 3. Pinecone vector DB
load_dotenv()
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name='law-index',
)

## 4. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  

## 4.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

## 4.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

## 5.0 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 #########

## 5.1. 프롬프트 작성

prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 5.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,)

## 5.3. LLM 모델 생성(chatOpenAI 인스턴스 생성)  
llm = ChatOpenAI(model='gpt-4o')

## 5.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message

c:\Users\71\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AIMessage(content='전세 사기 피해자에 대한 금융 지원은 주로 정부와 금융기관에서 마련한 특별 대출 프로그램과 지원 정책을 통해 이루어집니다. 한국에서는 전세 사기 피해를 입은 사람들을 돕기 위해 다양한 형태의 금융 지원이 제공됩니다. 여기에는 다음과 같은 지원책이 포함될 수 있습니다.\n\n1. **긴급 대출 지원**: 전세 사기 피해자가 당장 새로운 주거지를 구해야 하는 경우, 긴급 생계비나 주거비 대출을 저리로 제공받을 수 있습니다. 이를 통해 피해자는 새로 이사할 집을 마련할 수 있는 자금을 확보할 수 있습니다.\n\n2. **상환 유예**: 전세 사기를 당하여 경제적 어려움을 겪고 있는 피해자에 대해서는 기존의 채무 상환을 일정 기간 유예해주는 제도가 있을 수 있습니다. 이를 통해 경제적인 여유를 확보하고 사태를 해결할 시간을 벌 수 있습니다.\n\n3. **보증보험 활성화**: 임차인이 보증보험에 가입되어 있는 경우, 전세금을 돌려받지 못할 상황에 대비해 보험금을 청구할 수 있습니다. 정부는 이러한 보증보험의 가입을 권장하고, 피해자 지원을 위해 보험금을 신속하게 지급하는 체계를 마련할 수도 있습니다.\n\n4. **금융 상담 및 법률 지원**: 피해자들이 자신의 상황에 맞는 재정적 해결책을 찾을 수 있도록 금융 상담 서비스와 법률 상담을 제공받을 수 있습니다. 이를 통해 자신이 받을 수 있는 지원과 권리를 정확히 파악하고, 필요한 조치를 취할 수 있습니다.\n\n구체적인 지원 내용은 시기와 정책에 따라 변동될 수 있으므로, 가장 최신 정보를 확인하기 위해서는 정부의 관련 부서나 금융기관의 공지사항을 참고하거나, 법률 전문가와 상담하는 것이 중요합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 404, 'prompt_tokens': 60, 'total_tokens': 464, 'completion_tokens